In [4]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
# os.environ['JAVA_HOME']='/usr'
# os.environ['SPARK_HOME'] ='/usr/local/lib/python3.9/site-packages/pyspark/'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
# os.environ['HADOOP_USER_NAME']='skhalikov'
os.environ['HADOOP_USER_NAME']='student02'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import pyspark.sql.functions as F

In [7]:
conf = SparkConf().setAppName('Overwrite_partition_in_HDFS_20240513')\
    .setMaster('yarn') \
    .set('spark.default.parallelism', '4') \
    .set('spark.sql.shuffle.partitions', '100')\
    .set('spark.dynamicAllocation.enabled', 'false')\
    .set("spark.sql.sources.partitionOverwriteMode", "dynamic") 
 # Required for overwriting ONLY the required partitioned folders, and not the entire root folder

spark = SparkSession.builder.config(conf=conf).enableHiveSupport() \
    .getOrCreate()
# spark.sparkContext.setLogLevel('INFO')

sc = spark.sparkContext
print("app_id".ljust(50), sc.applicationId)
print("spark.sql.sources.partitionOverwriteMode".ljust(50), spark.conf.get("spark.sql.sources.partitionOverwriteMode"))

app_id                                             application_1711527309433_0231
spark.sql.sources.partitionOverwriteMode           dynamic


In [12]:
import random
from datetime import date, timedelta

end = date.today()
start = end + timedelta(days=-300)
length_customers = 10000

names = ['Bob', 'James', 'Marek', 'Johannes', 'Иванов', 'Смирнов', 'Кузнецов', 'Попов', 'Васильев', 'Петров', 'Соколов', 'Михайлов', 'Новиков', 'Федоров', 'Морозов', 'Волков', 'Алексеев', 'Лебедев', 'Семенов', 'Егоров', 'Павлов', 'Козлов', 'Степанов', 'Николаев']
countries = ['rus', 'bel', 'usa', 'fra', 'aus', 'ger']
customers = list()

save_path = "/user/student02/customers_partitioned_year_month"

In [18]:
for i in range(length_customers):
    random_date = start + (end - start) * random.random()
    customer = (names[random.randint(0, len(names)-1)], countries[random.randint(0, len(countries)-1)], random.randint(-1000000, 1000000), random_date)
    customers.append(customer)

df_customers = spark.createDataFrame(data=customers, schema=["name","country","balance","last_order_date"])\
    .withColumn("year", F.year("last_order_date"))\
    .withColumn("month", F.month("last_order_date"))

df_customers.groupBy("year", "month").count().orderBy("year", "month").show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|    7|  432|
|2023|    8| 1031|
|2023|    9|  984|
|2023|   10| 1020|
|2023|   11|  981|
|2023|   12| 1026|
|2024|    1| 1040|
|2024|    2|  933|
|2024|    3| 1077|
|2024|    4| 1030|
|2024|    5|  446|
+----+-----+-----+



In [19]:
df_customers.coalesce(1).write.partitionBy("year", "month").mode("overwrite")\
    .format("csv").save(save_path)

In [20]:
customers_add = list()

for i in range(5):
    # random_date = start + (end - start) * random.random()
    add_date = end
    customer = (names[random.randint(0, len(names)-1)], countries[random.randint(0, len(countries)-1)], random.randint(-1000000, 1000000), add_date)
    customers_add.append(customer)

df_customers_add = spark.createDataFrame(data=customers_add, schema=["name","country","balance","last_order_date"])\
    .withColumn("year", F.year("last_order_date"))\
    .withColumn("month", F.month("last_order_date"))

df_customers_add.groupBy("year", "month").count().orderBy("year", "month").show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2024|    5|    5|
+----+-----+-----+



In [22]:
df_customers_write = df_customers.union(df_customers_add).where("year = 2024 and month = 5")
df_customers_write.groupBy("year", "month").count().orderBy("year", "month").show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2024|    5|  451|
+----+-----+-----+



In [23]:
df_customers_write.coalesce(1).write.partitionBy("year", "month").mode("overwrite")\
    .format("csv").save(save_path)

In [24]:
df_customers_check = spark.read.csv(save_path).groupBy("year", "month").count().orderBy("year", "month").show()

+----+-----+-----+
|year|month|count|
+----+-----+-----+
|2023|    7|  432|
|2023|    8| 1031|
|2023|    9|  984|
|2023|   10| 1020|
|2023|   11|  981|
|2023|   12| 1026|
|2024|    1| 1040|
|2024|    2|  933|
|2024|    3| 1077|
|2024|    4| 1030|
|2024|    5|  451|
+----+-----+-----+



In [4]:
--

SyntaxError: invalid syntax (88891658.py, line 1)

In [25]:
spark.stop()